### Scripts to construct a water balance from LIS ouptut, for GRACE comparison and forward modeling

In [1]:
# Import the necessary libraries

%matplotlib inline
import os
import sys
import datetime

import xarray as xr

import pandas as pd

from dask.diagnostics import ProgressBar

import seaborn as sb

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

from himatpy.LIS import utils as LISutils

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
# Open all files into a single xarray dataset
ds = LISutils.get_xr_dataset('/att/pubrepo/hma_data/products/LIS-new/', fname=None, multiple_nc=True)

In [12]:
ds

<xarray.Dataset>
Dimensions:           (SnowIce_profiles: 3, SnowLiq_profiles: 3, SnowTProf_profiles: 3, SoilMoist_profiles: 4, SoilTemp_profiles: 4, east_west: 1896, north_south: 1696, time: 5478, z_soil_profiles: 4)
Coordinates:
  * time              (time) datetime64[ns] 2001-01-01 2001-01-02 2001-01-03 ...
Dimensions without coordinates: SnowIce_profiles, SnowLiq_profiles, SnowTProf_profiles, SoilMoist_profiles, SoilTemp_profiles, east_west, north_south, z_soil_profiles
Data variables:
    lat               (time, north_south, east_west) float64 nan nan nan nan ...
    lon               (time, north_south, east_west) float64 nan nan nan nan ...
    Swnet_tavg        (time, north_south, east_west) float64 nan nan nan nan ...
    Lwnet_tavg        (time, north_south, east_west) float64 nan nan nan nan ...
    Qle_tavg          (time, north_south, east_west) float64 nan nan nan nan ...
    Qh_tavg           (time, north_south, east_west) float64 nan nan nan nan ...
    Qg_tavg        

In [ ]:
## FIRST, searlize this to a single netcdf, retaining the daily resolution but keeping only these 
## variables and doing the unit adjustments:

# The variables we want are:

{'Qsm_tavg','Rainf_tavg','Qs_tavg','Snowf_tavg','Qsb_tavg','Evap_tavg','TWS_tavg'}

# The associated names are:
{'Snowmelt','Rainfall','SurfaceRunoff','Snowfall','SubsfcRunoff','Evapotranspiration','TWS'}

# multiplication factor for units

{864000,864000,864000,864000,864000,864000,0}

# attributes for the first 6 variables:

{'Daily <snowmelt> in units of mm we', '....'}

# attributes for the TWS:

{'Daily change in water storage'}

In [ ]:
## NEXT calculate monthly sums, e.g.:

with ProgressBar():
    TWS_monthly = ds['TWS_tavg'].resample('MS', 'time', how = 'sum')

# attributes for the first 6 variables:

{'Cumulative monthly <snowmelt> in units of mm we','Cumulative monthly ....'}

# attributes for TWS:

{'Change in monthly water storage'}
       
# then concatenate all into a single netcdf and serialize to /att/nobackup/aarendt    
    
        